In [1]:
import pandas as pd
import numpy as np
import collections
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [2]:
S1= 'sunshine state enjoy sunshine'
S2 ='brown fox jump high, brown fox run'
S3 ='sunshine state fox run fast'


In [16]:
#bagofWords usuing count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X = vectorizer.fit_transform([S1,S2,S3])
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
print('BAG of WORDS')
df_bow_sklearn



BAG of WORDS


,brown,enjoy,fast,fox,high,jump,run,state,sunshine
0,0,1,0,0,0,0,0,1,2
1,2,0,0,2,1,1,1,0,0
2,0,0,1,1,0,0,1,1,1


In [17]:
def bow(wordDict):
  bag= []
  for word, count in wordDict.items():
            if count>0:
              for x in range(count):
                bag.append(word)
  return bag

In [18]:
#making dictionaries of vector
S1dict=df_bow_sklearn.iloc[0].to_dict()
bowS1 =bow(S1dict)
S2dict=df_bow_sklearn.iloc[1].to_dict()
bowS2 = bow(S2dict)
S3dict=df_bow_sklearn.iloc[2].to_dict()
bowS3 = bow(S3dict)



In [19]:
#function to compute tf
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

tfBowS1 = computeTF(S1dict, bowS1)
tfBowS2 = computeTF(S2dict, bowS2)
tfBowS3 = computeTF(S3dict, bowS3)

In [21]:
#printing the tf
tf=pd.DataFrame([tfBowS1,tfBowS2,tfBowS3])
print("The TF values using function")
tf

The TF values using function


,brown,enjoy,fast,fox,high,jump,run,state,sunshine
0,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.5
1,0.285714,0.00,0.0,0.285714,0.142857,0.142857,0.142857,0.00,0.0
2,0.000000,0.00,0.2,0.200000,0.000000,0.000000,0.200000,0.20,0.2


In [22]:
#func to compute idf
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, count in doc.items():
            if count > 0:
                idfDict[word] += 1
    
    for word, count in idfDict.items():
        idfDict[word] = math.log10(N / float(count))
        
    return idfDict    
    

In [25]:
#finding idf
idfs = computeIDF([S1dict,S2dict,S3dict])
idfs_pd=pd.DataFrame([idfs], index=['idf'])
print("IDF of vocabulary using function")
idfs_pd

IDF of vocabulary using function


,brown,enjoy,fast,fox,high,jump,run,state,sunshine
idf,0.477121,0.477121,0.477121,0.176091,0.477121,0.477121,0.176091,0.176091,0.176091


In [26]:
#function for tf_idf
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, count in tfBow.items():
        tfidf[word] = count*idfs[word]
    return tfidf

In [27]:
#printing tfidf using function
tfidfS1 = computeTFIDF(tfBowS1, idfs)
tfidfS2 = computeTFIDF(tfBowS2, idfs)
tfidfS3 = computeTFIDF(tfBowS3, idfs)

tfid_pd= pd.DataFrame([tfidfS1, tfidfS2,tfidfS3], index=['S1','S2','S3'])
print("TF-IDF")
tfid_pd

TF-IDF


,brown,enjoy,fast,fox,high,jump,run,state,sunshine
S1,0.00000,0.11928,0.000000,0.000000,0.00000,0.00000,0.000000,0.044023,0.088046
S2,0.13632,0.00000,0.000000,0.050312,0.06816,0.06816,0.025156,0.000000,0.000000
S3,0.00000,0.00000,0.095424,0.035218,0.00000,0.00000,0.035218,0.035218,0.035218


In [29]:
#calculating tf using tfid_transformer
tf_transformer = TfidfTransformer(use_idf=False).fit(df_bow_sklearn)
tf_model = tf_transformer.transform(df_bow_sklearn)
df_tf = pd.DataFrame(tf_model.toarray(),columns=vectorizer.get_feature_names_out(), index=['S1','S2','S3'])
print('tf values using TTfidf Transformer')
df_tf

tf values using TTfidf Transformer


,brown,enjoy,fast,fox,high,jump,run,state,sunshine
S1,0.000000,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.816497
S2,0.603023,0.000000,0.000000,0.603023,0.301511,0.301511,0.301511,0.000000,0.000000
S3,0.000000,0.000000,0.447214,0.447214,0.000000,0.000000,0.447214,0.447214,0.447214


In [ ]:
#calculating idf using idf_transformer
idf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True).fit(X)
idf_model = idf_transformer.idf_
df_idf = pd.DataFrame(idf_model,index=vectorizer.get_feature_names_out() , columns=['IDF'])
df_idf


,IDF
brown,1.693147
enjoy,1.693147
fast,1.693147
fox,1.287682
high,1.693147
jump,1.693147
run,1.287682
state,1.287682
sunshine,1.287682


In [ ]:
#calculating tfidf using idf_transformer
tfidf_model = idf_transformer.transform(X)
print("TF.IDF values:", )

df_tfidf = pd.DataFrame(tfidf_model.toarray(),columns=vectorizer.get_feature_names_out() , index=['S1','S2','S3'])
df_tfidf

TF.IDF values:


,brown,enjoy,fast,fox,high,jump,run,state,sunshine
S1,0.000000,0.50689,0.000000,0.000000,0.000000,0.000000,0.000000,0.385503,0.771006
S2,0.670703,0.00000,0.000000,0.510087,0.335352,0.335352,0.255044,0.000000,0.000000
S3,0.000000,0.00000,0.549351,0.417796,0.000000,0.000000,0.417796,0.417796,0.417796


In [11]:
# import required libraries
import numpy as np
from numpy.linalg import norm
 
# define two lists or array
S1list=df_bow_sklearn.iloc[0].to_list()
S3list=df_bow_sklearn.iloc[2].to_list()



# compute cosine similarity
cosine = np.dot(S1list, S3list)/(norm(S1list)*norm(S3list))
print("Cosine Similarity between S1 and S3:", cosine)

Cosine Similarity between S1 and S3: 0.5477225575051661
